# Imports

In [1]:
import re
import unicodedata
import spacy
import pandas as pd

# Funções

In [2]:

def limpar_dados(filepath):
    df = pd.read_excel(filepath)
    
    # Normaliza os nomes das colunas
    df.columns = df.columns.str.strip().str.lower()
    
    # Seleciona as colunas de interesse
    colunas_necessarias = ['espécie', 'regiao_corpo_str', 'lista de imagens', 'laudo', 'laudo alterado']
    df = df[colunas_necessarias].copy()
    
    # Trata 'lista de imagens' para garantir que seja lista
    df['lista de imagens'] = df['lista de imagens'].apply(lambda x: eval(x) if isinstance(x, str) and x.startswith('[') else [img.strip() for img in x.split(',')])

    # Cria coluna 'prompt'
    def criar_prompt(row):
        alterado = "alterado" if row['laudo alterado'] == 1 else "sem alterações"
        return f"Este é um exame de raio-x de um {row['espécie']}, região: {row['regiao_corpo_str']}. Resultado: {alterado}."
    
    df['prompt'] = df.apply(criar_prompt, axis=1)
    
    return df


In [3]:
# Carregar modelo spaCy para português
nlp = spacy.load('pt_core_news_sm')

def limpar_laudo(texto):
    if pd.isnull(texto):
        return ""  # Trata valores nulos

    # 1. Minúsculas
    texto = texto.lower()

    # 2. Remover acentos
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode('utf-8')

    # 3. Remover expressões comuns
    expressoes_comuns = [
        'paciente:', 'exame:', 'observacao:', 'conclusao:',
        'laudo:', 'imagem:', 'descricao:', 'analise:'
    ]
    for exp in expressoes_comuns:
        texto = texto.replace(exp, '')

    # 4. Remover pontuação (exceto hífen)
    texto = re.sub(r'[^\w\s\-]', '', texto)

    # 5. Remover múltiplos espaços
    texto = re.sub(r'\s+', ' ', texto).strip()

    # 6. Lematização e remoção de stopwords com spaCy
    doc = nlp(texto)
    
    # Stopwords personalizadas
    stop_words = nlp.Defaults.stop_words
    palavras_importantes = {'sem', 'nao', 'com', 'presenca', 'ausencia'}
    stop_words = stop_words - palavras_importantes

    # Filtra tokens não stopwords e lematiza
    lemas = [token.lemma_ for token in doc if token.text not in stop_words]

    return ' '.join(lemas)


# Executando funções

In [4]:
df = limpar_dados("laudos_piloto002.xlsx")

In [5]:
df['laudo'] = df['laudo'].apply(limpar_laudo)

In [6]:
df.head(4)

,espécie,regiao_corpo_str,lista de imagens,laudo,laudo alterado,prompt
0,CANINO,COLUNA,[001_BALTAZAR ANDRESSA 12 anos 17225 29_04_202...,diminuicao espaco intervertebral t11-12 t13-l1...,0,"Este é um exame de raio-x de um CANINO, região..."
1,FELINO,TORAX,[002_ESMERALDA WILLIAN 14 anos 17185 28_04_202...,sem evidencia nodulo parenquima pulmonar detec...,1,"Este é um exame de raio-x de um FELINO, região..."
2,CANINO,EXTREMIDADES,[003_LUNA RENATA 10 anos 17231 29_04_2025_pg2_...,-proliferacao ossea solir regular adjacente fa...,1,"Este é um exame de raio-x de um CANINO, região..."
3,FELINO,ABDOMEN,[005_AIRAM WALTON 2 meses 17191 28_04_2025_pg2...,homogeneidade cavidade abdominal visibilizacao...,0,"Este é um exame de raio-x de um FELINO, região..."


In [7]:
df['espécie'].value_counts()

espécie
CANINO       684
FELINO       144
AVE           99
QUELONIO      23
SILVESTRE     14
QUILONIO       1
CAINO          1
Name: count, dtype: int64

In [8]:
df = df[df['espécie'].isin(['CANINO', 'FELINO'])]

In [9]:
df = df.drop(['espécie'], axis=1)

In [10]:
df['regiao_corpo_str'].value_counts()

regiao_corpo_str
TORAX           338
EXTREMIDADES    251
COLUNA          124
ABDOMEN          69
CRANIO           46
Name: count, dtype: int64

In [11]:
df['prompt'].head(1)

0    Este é um exame de raio-x de um CANINO, região...
Name: prompt, dtype: object

In [12]:
df['laudo_alterado'] = df['laudo alterado']

In [13]:
df.dropna()

,regiao_corpo_str,lista de imagens,laudo,laudo alterado,prompt,laudo_alterado
0,COLUNA,[001_BALTAZAR ANDRESSA 12 anos 17225 29_04_202...,diminuicao espaco intervertebral t11-12 t13-l1...,0,"Este é um exame de raio-x de um CANINO, região...",0
1,TORAX,[002_ESMERALDA WILLIAN 14 anos 17185 28_04_202...,sem evidencia nodulo parenquima pulmonar detec...,1,"Este é um exame de raio-x de um FELINO, região...",1
2,EXTREMIDADES,[003_LUNA RENATA 10 anos 17231 29_04_2025_pg2_...,-proliferacao ossea solir regular adjacente fa...,1,"Este é um exame de raio-x de um CANINO, região...",1
3,ABDOMEN,[005_AIRAM WALTON 2 meses 17191 28_04_2025_pg2...,homogeneidade cavidade abdominal visibilizacao...,0,"Este é um exame de raio-x de um FELINO, região...",0
4,TORAX,[007_LAURA ONELI 6 anos 16026 01_04_2025_pg2_i...,segmento vertebral cervical sem alteraco radio...,1,"Este é um exame de raio-x de um CANINO, região...",1
...,...,...,...,...,...,...
960,TORAX,[992_FRAJOLA RENILDA 3 anos 15837 29_03_2025_p...,sem evidencia nodulo parenquima pulmonar detec...,1,"Este é um exame de raio-x de um FELINO, região...",1
962,TORAX,[995_NICK TOMAS 14 anos 15876 29_03_2025_pg2_i...,presenca areo arredondada radiopacidade tecido...,1,"Este é um exame de raio-x de um CANINO, região...",1
963,TORAX,[996_SHEIK ANA SILVIA 9 anos 15874 28_03_2025_...,achar radiografico opacificacao focal padrao a...,1,"Este é um exame de raio-x de um CANINO, região...",1
964,EXTREMIDADES,[998_BIZU CANUTO 8 anos 15421 28_03_2025_pg2_i...,sutil linha radiotransparente terco distal dia...,0,"Este é um exame de raio-x de um CANINO, região...",0


In [14]:
df = df.drop('laudo_alterado', axis=1)

In [15]:
df[df['regiao_corpo_str'] == 'TORAX'].to_excel('excel para aprendizado/torax_para_aprendizado.xlsx', index=False)

In [16]:
df[df['regiao_corpo_str'] == 'EXTREMIDADES'].to_excel('excel para aprendizado/extremidades_para_aprendizado.xlsx', index=False)

In [17]:
df[df['regiao_corpo_str'] == 'COLUNA'].to_excel('excel para aprendizado/coluna_para_aprendizado.xlsx', index=False)

In [18]:
df[df['regiao_corpo_str'] == 'CRANIO'].to_excel('excel para aprendizado/cranio_para_aprendizado.xlsx', index=False)

In [19]:
df[df['regiao_corpo_str'] == 'ABDOMEN'].to_excel('excel para aprendizado/abdomen_para_aprendizado.xlsx', index=False)

In [20]:
df[df['regiao_corpo_str'] == 'EXTREMIDADES, COLUNA'].to_excel('excel para aprendizado/extrem_col_para_aprendizado.xlsx', index=False)

In [21]:
df.shape

(828, 5)

In [22]:
df.to_excel('laudos_piloto003.xlsx', index=False)